# Base table creation 
## Prerequesites
This is the second approach based on the fundings of our fellow machine learner.   

We need a clustering in time slots to predict based on that. This is due the fact that activity prediction does not work based on activity. We need another feature called time slot.

So further inspection is needed and forming of new feature (time_slot). 

# Setup

In [1]:
# dependencies
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline 

import seaborn as sns

import missingno as msno

# Get data

In [2]:
df = pd.read_csv('data/clean_table.csv')

# Data Exploration
## Standard analysis

In [3]:
df.shape

(3386, 7)

In [4]:
df.dtypes

activity               object
category               object
start_date[ms]          int64
start_date             object
end_date[ms]            int64
end_date               object
activityDuration[m]     int64
dtype: object

In [5]:
df.head()

,activity,category,start_date[ms],start_date,end_date[ms],end_date,activityDuration[m]
0,Trello,Personal Adjusting,1540159273005,Mon Oct 22 00:01:13 GMT+02:00 2018,1540159869559,Mon Oct 22 00:11:09 GMT+02:00 2018,9
1,Series / Docu,Entertainment,1540159869559,Mon Oct 22 00:11:09 GMT+02:00 2018,1540162820068,Mon Oct 22 01:00:20 GMT+02:00 2018,49
2,Sleep,Refresh,1540162820068,Mon Oct 22 01:00:20 GMT+02:00 2018,1540189458018,Mon Oct 22 08:24:18 GMT+02:00 2018,443
3,Moving - youtube,Transport,1540189458018,Mon Oct 22 08:24:18 GMT+02:00 2018,1540189949037,Mon Oct 22 08:32:29 GMT+02:00 2018,8
4,Trello,Personal Adjusting,1540189949037,Mon Oct 22 08:32:29 GMT+02:00 2018,1540190444165,Mon Oct 22 08:40:44 GMT+02:00 2018,8


## Deeper exploration 
Timestamp given here are unix but 13 digits instead of 10. Means we have to transform the timestamp.

### Deeper exploration - weekly 

In [6]:
def get_data_by(data, timestamp_start, timestamp_end):
    return data[(data['start_date[ms]']  >= timestamp_start) & 
                (data['end_date[ms]']  <= timestamp_end)]


In [7]:
start = 1541159273005
end =   1551373190000
get_data_by(df, start, end)

,activity,category,start_date[ms],start_date,end_date[ms],end_date,activityDuration[m]
355,WC,Personal Hygiene,1541161421218,Fri Nov 02 13:23:41 GMT+01:00 2018,1541162184135,Fri Nov 02 13:36:24 GMT+01:00 2018,12
356,Learning - university,Learning,1541162184135,Fri Nov 02 13:36:24 GMT+01:00 2018,1541168809917,Fri Nov 02 15:26:49 GMT+01:00 2018,110
357,WC,Personal Hygiene,1541168809917,Fri Nov 02 15:26:49 GMT+01:00 2018,1541169448687,Fri Nov 02 15:37:28 GMT+01:00 2018,10
358,Social - Close,Social - Persist,1541169448687,Fri Nov 02 15:37:28 GMT+01:00 2018,1541170048687,Fri Nov 02 15:47:28 GMT+01:00 2018,10
359,Learning - university,Learning,1541170048687,Fri Nov 02 15:47:28 GMT+01:00 2018,1541173112283,Fri Nov 02 16:38:32 GMT+01:00 2018,51
360,Learning - university,Learning,1541173112283,Fri Nov 02 16:38:32 GMT+01:00 2018,1541174289201,Fri Nov 02 16:58:09 GMT+01:00 2018,19
361,Moving - social,Transport,1541174289201,Fri Nov 02 16:58:09 GMT+01:00 2018,1541180721517,Fri Nov 02 18:45:21 GMT+01:00 2018,107
362,Food,Feeding,1541180721517,Fri Nov 02 18:45:21 GMT+01:00 2018,1541182380717,Fri Nov 02 19:13:00 GMT+01:00 2018,27
363,Social - Real Life,Social - Persist,1541182380717,Fri Nov 02 19:13:00 GMT+01:00 2018,1541183108746,Fri Nov 02 19:25:08 GMT+01:00 2018,12
364,Moving - social,Transport,1541183108746,Fri Nov 02 19:25:08 GMT+01:00 2018,1541186792387,Fri Nov 02 20:26:32 GMT+01:00 2018,61


#### Get timestamps per week

In [8]:
import datetime
    
def get_timestamps_for_week(weeknumber,year):
    week = "{}-W{}".format(year,weeknumber)
    date = datetime.datetime.strptime(week + '-1', "%Y-W%W-%w")
    
    
    start_timestamp = date.timestamp()*1000
    end_timestamp = (date + datetime.timedelta(days=7) ).timestamp()*1000
    return start_timestamp,end_timestamp

In [9]:
week = 1
year = 2019
timestamps_week = get_timestamps_for_week(week,year)
timestamps_week

(1546815600000.0, 1547420400000.0)

In [10]:
get_data_by(df,timestamps_week[0],timestamps_week[1])

,activity,category,start_date[ms],start_date,end_date[ms],end_date,activityDuration[m]
2231,Playing,Entertainment,1546815714657,Mon Jan 07 00:01:54 GMT+01:00 2019,1546816974657,Mon Jan 07 00:22:54 GMT+01:00 2019,21
2232,Sleep,Refresh,1546816974657,Mon Jan 07 00:22:54 GMT+01:00 2019,1546847289279,Mon Jan 07 08:48:09 GMT+01:00 2019,505
2233,Clean,Personal Hygiene,1546847289279,Mon Jan 07 08:48:09 GMT+01:00 2019,1546847949279,Mon Jan 07 08:59:09 GMT+01:00 2019,11
2234,Food,Feeding,1546847949279,Mon Jan 07 08:59:09 GMT+01:00 2019,1546849689279,Mon Jan 07 09:28:09 GMT+01:00 2019,29
2235,Wasted - Moving,Wasted Time,1546849689279,Mon Jan 07 09:28:09 GMT+01:00 2019,1546851309279,Mon Jan 07 09:55:09 GMT+01:00 2019,27
2236,Work - Task planning,Value Contribution,1546851309279,Mon Jan 07 09:55:09 GMT+01:00 2019,1546853222748,Mon Jan 07 10:27:02 GMT+01:00 2019,31
2237,Work - Purpose,Purpose,1546853222748,Mon Jan 07 10:27:02 GMT+01:00 2019,1546858022748,Mon Jan 07 11:47:02 GMT+01:00 2019,80
2238,Work - Meeting,Value Contribution,1546858022748,Mon Jan 07 11:47:02 GMT+01:00 2019,1546861322748,Mon Jan 07 12:42:02 GMT+01:00 2019,55
2239,Work - Social,Value Contribution,1546861322748,Mon Jan 07 12:42:02 GMT+01:00 2019,1546861622748,Mon Jan 07 12:47:02 GMT+01:00 2019,5
2240,Food - shit,Feeding,1546861622748,Mon Jan 07 12:47:02 GMT+01:00 2019,1546861982748,Mon Jan 07 12:53:02 GMT+01:00 2019,6


### Deeper exploration - timeslot 

Define time_slot feature:  
- We have monday till sunday
- every half hour is a slot
--> This assumptions lead to 

time_slot : t   
t0 = Monday 00:00-00:29    
t1 = Monday 00:30-00:59     
...   
t332 = Sunday 23:45-23:59

Goal is to create a mapping for the time slots
```python
mapping = {
        'Mon': {
            '00': {
                '00': [0,1,2,...,29],
                '30': [30,31,32,...,59]
            },
            '01': {
                '00': [0,1,2,...,29],
                '30': [30,31,32,...,59]
            }, ...
        }, ...
    }
```

In [11]:
mapping = {}
timeslot = 0

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
hours = [h for h in range(0,24)]
# minutes = [str(m) for m in range(1,60,30)] <-- missing 0
minutes_slot = [[x for x in range(0,30)],[x for x in range(30,60)]]

for day in days:
    mapping[day] = {}
    for hour in hours:
        mapping[day][hour] = {}
        for minute_slot in minutes_slot:
            for minute in minute_slot:
                mapping[day][hour][minute] = timeslot
            timeslot += 1

In [12]:
def get_timeslot(start_date, end_date):
    
    # extract the days TODO
    start_day = start_date[0:3]
    end_day = end_date[0:3]
    
    # extract the time
    start_hour = int(start_date[11:13])
    start_minute = int(start_date[14:16])
    end_hour = int(end_date[11:13])
    end_minute = int(end_date[14:16])
    
    # map to right timeslot
    timeslot_start = mapping[start_day][start_hour][start_minute]
    timeslot_end = mapping[end_day][end_hour][end_minute]
    timeslots = [x for x in range(timeslot_start, timeslot_end+1)]
    return timeslots

# just for test purposes
# start_date = 'Mon Oct 22 00:00:20 GMT+02:00 2018'
# end_date = 'Mon Oct 22 23:31:09 GMT+02:00 2018'
# get_timeslot(start_date, end_date)

# Base table creation

## Feature engineering 
Let's take only relevant features into our base table:

1. id
- activity_name,
- category_name,
- timeslot
- start_date,
- start_date_timestamp,
- end_date,
- end_date_timestamp,


In [13]:
df.dtypes

activity               object
category               object
start_date[ms]          int64
start_date             object
end_date[ms]            int64
end_date               object
activityDuration[m]     int64
dtype: object

In [14]:
df

,activity,category,start_date[ms],start_date,end_date[ms],end_date,activityDuration[m]
0,Trello,Personal Adjusting,1540159273005,Mon Oct 22 00:01:13 GMT+02:00 2018,1540159869559,Mon Oct 22 00:11:09 GMT+02:00 2018,9
1,Series / Docu,Entertainment,1540159869559,Mon Oct 22 00:11:09 GMT+02:00 2018,1540162820068,Mon Oct 22 01:00:20 GMT+02:00 2018,49
2,Sleep,Refresh,1540162820068,Mon Oct 22 01:00:20 GMT+02:00 2018,1540189458018,Mon Oct 22 08:24:18 GMT+02:00 2018,443
3,Moving - youtube,Transport,1540189458018,Mon Oct 22 08:24:18 GMT+02:00 2018,1540189949037,Mon Oct 22 08:32:29 GMT+02:00 2018,8
4,Trello,Personal Adjusting,1540189949037,Mon Oct 22 08:32:29 GMT+02:00 2018,1540190444165,Mon Oct 22 08:40:44 GMT+02:00 2018,8
5,Moving - youtube,Transport,1540190444165,Mon Oct 22 08:40:44 GMT+02:00 2018,1540190532145,Mon Oct 22 08:42:12 GMT+02:00 2018,1
6,Clean,Personal Hygiene,1540190532145,Mon Oct 22 08:42:12 GMT+02:00 2018,1540191903726,Mon Oct 22 09:05:03 GMT+02:00 2018,22
7,Food,Feeding,1540191903726,Mon Oct 22 09:05:03 GMT+02:00 2018,1540193553126,Mon Oct 22 09:32:33 GMT+02:00 2018,27
8,Moving - youtube,Transport,1540193553126,Mon Oct 22 09:32:33 GMT+02:00 2018,1540194820783,Mon Oct 22 09:53:40 GMT+02:00 2018,21
9,Moving - youtube,Transport,1540194820783,Mon Oct 22 09:53:40 GMT+02:00 2018,1540195000783,Mon Oct 22 09:56:40 GMT+02:00 2018,3


In [15]:
base_table = pd.DataFrame()
base_table['id'] = [x for x in range(len(df))]
base_table['activity_name'] = df['activity']
base_table['category_name'] = df['category']
# fix empty timeslots
base_table['timeslot'] = [get_timeslot(x,y) for x,y in zip(df['start_date'],df['end_date'])]
base_table['start_date'] = df['start_date']
base_table['start_date_timestamp'] = df['start_date[ms]']
base_table['end_date'] = df['end_date']
base_table['end_date_timestamp'] = df['end_date[ms]']
base_table

,id,activity_name,category_name,timeslot,start_date,start_date_timestamp,end_date,end_date_timestamp
0,0,Trello,Personal Adjusting,[0],Mon Oct 22 00:01:13 GMT+02:00 2018,1540159273005,Mon Oct 22 00:11:09 GMT+02:00 2018,1540159869559
1,1,Series / Docu,Entertainment,"[0, 1, 2]",Mon Oct 22 00:11:09 GMT+02:00 2018,1540159869559,Mon Oct 22 01:00:20 GMT+02:00 2018,1540162820068
2,2,Sleep,Refresh,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",Mon Oct 22 01:00:20 GMT+02:00 2018,1540162820068,Mon Oct 22 08:24:18 GMT+02:00 2018,1540189458018
3,3,Moving - youtube,Transport,"[16, 17]",Mon Oct 22 08:24:18 GMT+02:00 2018,1540189458018,Mon Oct 22 08:32:29 GMT+02:00 2018,1540189949037
4,4,Trello,Personal Adjusting,[17],Mon Oct 22 08:32:29 GMT+02:00 2018,1540189949037,Mon Oct 22 08:40:44 GMT+02:00 2018,1540190444165
5,5,Moving - youtube,Transport,[17],Mon Oct 22 08:40:44 GMT+02:00 2018,1540190444165,Mon Oct 22 08:42:12 GMT+02:00 2018,1540190532145
6,6,Clean,Personal Hygiene,"[17, 18]",Mon Oct 22 08:42:12 GMT+02:00 2018,1540190532145,Mon Oct 22 09:05:03 GMT+02:00 2018,1540191903726
7,7,Food,Feeding,"[18, 19]",Mon Oct 22 09:05:03 GMT+02:00 2018,1540191903726,Mon Oct 22 09:32:33 GMT+02:00 2018,1540193553126
8,8,Moving - youtube,Transport,[19],Mon Oct 22 09:32:33 GMT+02:00 2018,1540193553126,Mon Oct 22 09:53:40 GMT+02:00 2018,1540194820783
9,9,Moving - youtube,Transport,[19],Mon Oct 22 09:53:40 GMT+02:00 2018,1540194820783,Mon Oct 22 09:56:40 GMT+02:00 2018,1540195000783


## Argument with dummy variables for activites 

In [16]:
dummy_columns = ['activity_name']
base_table = pd.get_dummies(base_table, columns=dummy_columns) 
base_table['activity_name'] = df['activity'] # will be removed by dummies. so we add it again
print(base_table.dtypes)

id                                             int64
category_name                                 object
timeslot                                      object
start_date                                    object
start_date_timestamp                           int64
end_date                                      object
end_date_timestamp                             int64
activity_name_Analyzing                        uint8
activity_name_Beatbox                          uint8
activity_name_Better Now                       uint8
activity_name_Bike                             uint8
activity_name_Celebration - Bar                uint8
activity_name_Celebration - Club               uint8
activity_name_Chess                            uint8
activity_name_Clean                            uint8
activity_name_Cut hair                         uint8
activity_name_Dance                            uint8
activity_name_Drink - Shit                     uint8
activity_name_Enjoying the moment             

# Conclusions TODO

In [17]:
base_table.head()

,id,category_name,timeslot,start_date,start_date_timestamp,end_date,end_date_timestamp,activity_name_Analyzing,activity_name_Beatbox,activity_name_Better Now,...,activity_name_Work - Meeting,activity_name_Work - Org,activity_name_Work - Purpose,activity_name_Work - Social,activity_name_Work - Task planning,activity_name_Work - wasted,activity_name_Youtube,activity_name_moving - groceries,activity_name_moving - eat,activity_name_untracked
0,0,Personal Adjusting,[0],Mon Oct 22 00:01:13 GMT+02:00 2018,1540159273005,Mon Oct 22 00:11:09 GMT+02:00 2018,1540159869559,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Entertainment,"[0, 1, 2]",Mon Oct 22 00:11:09 GMT+02:00 2018,1540159869559,Mon Oct 22 01:00:20 GMT+02:00 2018,1540162820068,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Refresh,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",Mon Oct 22 01:00:20 GMT+02:00 2018,1540162820068,Mon Oct 22 08:24:18 GMT+02:00 2018,1540189458018,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Transport,"[16, 17]",Mon Oct 22 08:24:18 GMT+02:00 2018,1540189458018,Mon Oct 22 08:32:29 GMT+02:00 2018,1540189949037,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Personal Adjusting,[17],Mon Oct 22 08:32:29 GMT+02:00 2018,1540189949037,Mon Oct 22 08:40:44 GMT+02:00 2018,1540190444165,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save clean table

In [18]:
base_table.to_csv('data/base_table_time_slots.csv', index='id')

In [19]:
base_table.head()

,id,category_name,timeslot,start_date,start_date_timestamp,end_date,end_date_timestamp,activity_name_Analyzing,activity_name_Beatbox,activity_name_Better Now,...,activity_name_Work - Meeting,activity_name_Work - Org,activity_name_Work - Purpose,activity_name_Work - Social,activity_name_Work - Task planning,activity_name_Work - wasted,activity_name_Youtube,activity_name_moving - groceries,activity_name_moving - eat,activity_name_untracked
0,0,Personal Adjusting,[0],Mon Oct 22 00:01:13 GMT+02:00 2018,1540159273005,Mon Oct 22 00:11:09 GMT+02:00 2018,1540159869559,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Entertainment,"[0, 1, 2]",Mon Oct 22 00:11:09 GMT+02:00 2018,1540159869559,Mon Oct 22 01:00:20 GMT+02:00 2018,1540162820068,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Refresh,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",Mon Oct 22 01:00:20 GMT+02:00 2018,1540162820068,Mon Oct 22 08:24:18 GMT+02:00 2018,1540189458018,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Transport,"[16, 17]",Mon Oct 22 08:24:18 GMT+02:00 2018,1540189458018,Mon Oct 22 08:32:29 GMT+02:00 2018,1540189949037,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Personal Adjusting,[17],Mon Oct 22 08:32:29 GMT+02:00 2018,1540189949037,Mon Oct 22 08:40:44 GMT+02:00 2018,1540190444165,0,0,0,...,0,0,0,0,0,0,0,0,0,0
